In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torch.utils.data.dataset as dataset
import numpy as np

from dataset import HealthcareDataset
from model import initialize_model

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Load trained model
net = initialize_model()

SAVED_MODEL_PATH = 'checkpoints/mobilenet_3_2'
net.load_state_dict(torch.load(SAVED_MODEL_PATH))

# Freeze layers
for param in net.parameters():
    param.requires_grad = False
    
# We just want to apply the feature extractor for now
net.classifier = nn.Identity()
net.to(device)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(9, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [4]:
# Get dataset
DATA_FOLDER = 'E:/xplore_data/data/'
HEALTH_FILE = 'data/dhs_gps.csv'
dimages = HealthcareDataset(DATA_FOLDER, HEALTH_FILE)
dimagesloader = torch.utils.data.DataLoader(dimages, batch_size=64, shuffle=False, num_workers=0)

In [5]:
# Apply feature extractor to the dataset
n = len(dimages)
extracted_features = torch.zeros(n, 1280+11)
i = 0
# Iterate over data.
for x, _, z in dimagesloader:
    x = x.to(device)
    j = i + x.shape[0]
    with torch.set_grad_enabled(False):
        outputs = net(x)
        extracted_features[i:j, :1280] = torch.squeeze(outputs).cpu()
        extracted_features[i:j, 1280:] = z
    i += 16

In [6]:
torch.isnan(extracted_features).nonzero()

tensor([], size=(0, 2), dtype=torch.int64)

In [7]:
dfeatures = dataset.TensorDataset(extracted_features)

# Randomly split this dataset into train and test.
dtrain, dtest = dataset.random_split(dfeatures, (n - (n//5), n//5))

In [8]:
# Simple neural network to predict healthcare
predictor = nn.Sequential(
    nn.Linear(1280, 11),
    nn.Sigmoid()
)
predictor.to(device)

Sequential(
  (0): Linear(in_features=1280, out_features=11, bias=True)
  (1): Sigmoid()
)

In [9]:
# Train the predictor model with MSE loss
def train_model(model, dataloader, optimizer, scheduler, num_epochs=4, test_loader=None):
    for epoch in range(num_epochs):
        print('Epoch {}/{}: '.format(epoch, num_epochs - 1), end='')

        model.train()
        running_loss = 0.0
        
        criterion = nn.MSELoss()

        # Iterate over data.
        for rows in dataloader:
            x = rows[0][:, :1280].to(device)
            y = rows[0][:, 1280:].to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                outputs = model(x)
                loss = criterion(outputs, y)
                loss.backward()
                optimizer.step()
                if scheduler is not None:
                    scheduler.step()

            # statistics
            running_loss += loss.item() * x.size(0)

        epoch_loss = running_loss / len(dtrain)
        print('Training loss: {:.4f}; '.format(epoch_loss), end='')
        test_loss = 0.0
        
        if test_loader is not None:
            for rows in test_loader:
                x = rows[0][:, :1280].to(device)
                y = rows[0][:, 1280:].to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(False):
                    outputs = model(x)
                    loss = criterion(outputs, y)

                # statistics
                test_loss += loss.item() * x.size(0)
        test_loss /= len(dtrain)
        print('Test loss: {:.4f}'.format(test_loss))

    return model


In [10]:
BATCH_SIZE = 32
EPOCHS = 30

# Data loader
dloader = torch.utils.data.DataLoader(dtrain, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
dtestloader = torch.utils.data.DataLoader(dtest, batch_size=512, shuffle=False, num_workers=0)
# Create training optimizer
optimizer = optim.Adam(predictor.parameters(), lr=0.01, weight_decay=0.01)


In [11]:
predictor = train_model(predictor, dloader, optimizer, scheduler=None, 
                        num_epochs=EPOCHS, test_loader=dtestloader)

Epoch 0/29: Training loss: 0.1927; Test loss: 0.0428
Epoch 1/29: Training loss: 0.1531; Test loss: 0.0367
Epoch 2/29: Training loss: 0.1330; Test loss: 0.0312
Epoch 3/29: Training loss: 0.1127; Test loss: 0.0286
Epoch 4/29: Training loss: 0.1090; Test loss: 0.0275
Epoch 5/29: Training loss: 0.1083; Test loss: 0.0255
Epoch 6/29: Training loss: 0.1018; Test loss: 0.0254
Epoch 7/29: Training loss: 0.0998; Test loss: 0.0255
Epoch 8/29: Training loss: 0.0990; Test loss: 0.0255
Epoch 9/29: Training loss: 0.1018; Test loss: 0.0251
Epoch 10/29: Training loss: 0.1007; Test loss: 0.0250
Epoch 11/29: Training loss: 0.1034; Test loss: 0.0263
Epoch 12/29: Training loss: 0.1009; Test loss: 0.0259
Epoch 13/29: Training loss: 0.1034; Test loss: 0.0254
Epoch 14/29: Training loss: 0.1000; Test loss: 0.0249
Epoch 15/29: Training loss: 0.0988; Test loss: 0.0258
Epoch 16/29: Training loss: 0.0995; Test loss: 0.0256
Epoch 17/29: Training loss: 0.0996; Test loss: 0.0258
Epoch 18/29: Training loss: 0.1015; Te

In [12]:
# Compute r-squared for training and test set
with torch.no_grad():
    split = [dtrain.indices, dtest.indices]
    tensor_inputs = [extracted_features[x, :1280].to(device) for x in split]
    outputs = [extracted_features[x, 1280:].numpy() for x in split]
    preds = [predictor(x).cpu().numpy() for x in tensor_inputs]

In [13]:
from sklearn.metrics import r2_score
for i in range(11):
    r2s = [r2_score(y[:, i], pred[:, i]) for y, pred in zip(outputs, preds)]
    print('%.4f, %.4f' % tuple(r2s))

0.1450, -0.1128
-0.1853, -0.5618
0.0964, -0.1023
-0.0292, -0.2340
-0.2355, -0.4630
-0.0240, -0.4437
0.4381, 0.2767
0.3065, 0.1315
-0.2450, -0.3509
0.1373, -0.0885
0.4426, 0.2798
